In [55]:
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import R2Score
from tensorflow.keras.losses import MeanSquaredError
import pandas as pd 
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score,root_mean_squared_error
import mysql.connector

In [56]:
mydb=mysql.connector.connect(host="127.0.0.1",user="root", passwd="Govinda457", database='leftovers1')
my=mydb.cursor(buffered=True)

In [57]:
my.callproc('total_orders')

()

In [58]:
results=my.execute('call item_call(101)',multi=True)
data=[]
try:
    for i in results:
        data=i.fetchall()
except:
    pass

In [59]:
data=pd.DataFrame(data,columns=['order_date','order_hour','quantity'])
data['order_day']=list(map((lambda x:x.weekday()),data['order_date'].to_list()))
scaler=StandardScaler()
data[['order_day','order_hour']]=scaler.fit_transform(data[['order_day','order_hour']])

In [69]:
x_train,x_test,y_train,y_test=train_test_split(data[['order_day','order_hour']],data['quantity'],test_size=0.2,shuffle=True)

In [70]:
linear=linear_model.LinearRegression()
reg=linear.fit(x_train,y_train)
result=reg.predict(x_test)
result=result.round(0)

In [71]:
print(r2_score(y_test,result))
print(root_mean_squared_error(y_test,result))

-0.4400780001625004
0.9150418500274112


In [63]:
model=Sequential()

In [64]:
model.add(Input((2,)))
model.add(Dense(2,activation='sigmoid'))
model.add(Dense(8,activation='sigmoid'))
model.add(Dense(1,activation='linear'))

In [65]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 2)              │             6 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 8)              │            24 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 39 (156.00 B)

 Trainable params: 39 (156.00 B)

 Non-trainable params: 0 (0.00 B)

In [66]:
model.compile(optimizer=Adam(),loss=MeanSquaredError(),metrics=[R2Score()])
model.fit(x_train,y_train,epochs=100)

Epoch 1/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 968us/step - loss: 1.0915 - r2_score: -0.8610
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 852us/step - loss: 0.8076 - r2_score: -0.3797
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 881us/step - loss: 0.7029 - r2_score: -0.2063
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 852us/step - loss: 0.6832 - r2_score: -0.1035
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 822us/step - loss: 0.6686 - r2_score: -0.0459
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 862us/step - loss: 0.6971 - r2_score: -0.0305
Epoch 7/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 920us/step - loss: 0.6501 - r2_score: -0.0211
Epoch 8/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 851us/step - loss: 0.5953 - r2_score: -0.0193
Epoch 9/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 908us/step - loss: 0.6739 - r2_score: -0.0154
Epoch 10/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.7092 - r2_score: -0.0129 
Epoch 11/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6749 - r2_score: -0.0194 
Epoch 12/100
32/32 ━━━━━━━━━━━━━

In [67]:
result=model.predict(x_test)

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


In [68]:
result=result.flatten()
r2_score(y_test,result)

-0.014117598533630371